In [2]:
import pandas as pd
import importlib
import os
import sys
import yfinance as yf
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [3]:
# Create some sample data to try now 
config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

filename_to_classname = {
    'larry_williams_price_action': 'LarryWilliamsPriceAction',
    'macd_bollinger_bands_mean_reversion': 'MACDBollingerBandsMeanReversion',
    'michael_harris_price_action' : 'MichaelHarrisPriceAction',
    'rsi_divergence': 'RSIDivergence',
    'scalping': 'Scalping',
    'volume_spike_reversal': 'VolumeSpikeReversal'
}

strategy_map = pd.read_csv("../data/processed/asset_strategies.csv").set_index("asset")
strategy_map

,weight,strategy,return
asset,,,
CZR,0.195536,Scalping,4.694043
INTC,0.159487,MeanReversion,29.333193
MHK,0.115818,MeanReversion,14.363444
BLDR,0.107738,RSIDivergence,25.506435
URI,0.088408,MichaelHarrisPriceAction,14.858904
ON,0.070599,RSIDivergence,38.940199
NCLH,0.031879,LarryWilliamsPriceAction,58.419585
ALB,0.028309,RSIDivergence,41.886634
VST,0.005340,LarryWilliamsPriceAction,122.958725


First function retrieves data from yf 

In [5]:
def get_asset_df(strategy_map, start, end):
    df_list = []
    for asset_name in strategy_map.index:
        # Retrieve yf finance data 
        df = yf.Ticker(asset_name).history(start=start, end=end, actions=False)

        # Align date formatting 
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

        df['Asset'] = asset_name
        df_list.append(df.reset_index())

    df_all = pd.concat(df_list).set_index(['Date', 'Asset']).sort_index()
    return df_all 

In [6]:
asset_df = get_asset_df(strategy_map=strategy_map, start= "2025-01-01", end="2025-03-01")

asset_df.head(10)

index        Open        High         Low       Close  \
Date       Asset                                                             
2025-01-01 AVAX-USD      0   35.690929   37.869911   34.960541   37.693439   
           LINK-USD      0   20.001163   21.771200   19.707378   21.674845   
           SOL-USD       0  189.266922  194.818497  187.879456  193.873734   
2025-01-02 ALB           0   85.956465   87.497790   83.937822   84.753235   
           AVAX-USD      1   37.693447   40.501595   37.689964   39.229160   
           BLDR          0  145.089996  146.029999  141.149994  141.979996   
           CZR           0   33.660000   34.084999   32.349998   32.590000   
           INTC          0   20.230000   20.430000   20.000000   20.219999   
           LINK-USD      1   21.674786   22.829363   21.672688   22.040438   
           MHK           0  120.370003  120.629997  115.830002  115.919998   

                         Volume  
Date       Asset                 
2025-01-01 AVAX-USD   334807162  
           LINK-USD   670017230  
           SOL-USD   2324231668  
2025-01-02 ALB          1866100  
           AVAX-USD   522067297  
           BLDR          906800  
           CZR          4674200  
           INTC        46438500  
           LINK-USD   893194042  
           MHK           572700

Second function implements AR model to derive expected return 

In [7]:
def compute_returns(asset_df):
    asset_df["Return"] = asset_df.groupby("Asset")["Close"].pct_change()
    return asset_df

asset_df_with_returns = compute_returns(asset_df)

In [8]:
asset_df_with_returns.head(10)

index        Open        High         Low       Close  \
Date       Asset                                                             
2025-01-01 AVAX-USD      0   35.690929   37.869911   34.960541   37.693439   
           LINK-USD      0   20.001163   21.771200   19.707378   21.674845   
           SOL-USD       0  189.266922  194.818497  187.879456  193.873734   
2025-01-02 ALB           0   85.956465   87.497790   83.937822   84.753235   
           AVAX-USD      1   37.693447   40.501595   37.689964   39.229160   
           BLDR          0  145.089996  146.029999  141.149994  141.979996   
           CZR           0   33.660000   34.084999   32.349998   32.590000   
           INTC          0   20.230000   20.430000   20.000000   20.219999   
           LINK-USD      1   21.674786   22.829363   21.672688   22.040438   
           MHK           0  120.370003  120.629997  115.830002  115.919998   

                         Volume    Return  
Date       Asset                           
2025-01-01 AVAX-USD   334807162       NaN  
           LINK-USD   670017230       NaN  
           SOL-USD   2324231668       NaN  
2025-01-02 ALB          1866100       NaN  
           AVAX-USD   522067297  0.040742  
           BLDR          906800       NaN  
           CZR          4674200       NaN  
           INTC        46438500       NaN  
           LINK-USD   893194042  0.016867  
           MHK           572700       NaN

In [9]:
def generate_expected_returns(data, ar_results):
    """ Calculate expected returns using fitted AR models """
    new_data = data.reset_index().merge(ar_results, on="Asset", how="left").set_index(["Date", "Asset"])
    new_data["Expected Return"] = new_data["const"] + new_data["Return"] * new_data["ar.L1"]
    return new_data

In [10]:
# Apply AR model
asset_df_with_AR = generate_expected_returns(asset_df_with_returns, ar_model_info)
asset_df_with_AR = asset_df_with_AR.drop(['Best Model','BIC', 'const','ar.L1', 'sigma2'], axis=1)
asset_df_with_AR

index        Open        High         Low       Close  \
Date       Asset                                                             
2025-01-01 AVAX-USD      0   35.690929   37.869911   34.960541   37.693439   
           LINK-USD      0   20.001163   21.771200   19.707378   21.674845   
           SOL-USD       0  189.266922  194.818497  187.879456  193.873734   
2025-01-02 ALB           0   85.956465   87.497790   83.937822   84.753235   
           AVAX-USD      1   37.693447   40.501595   37.689964   39.229160   
...                    ...         ...         ...         ...         ...   
2025-02-28 NCLH         38   23.540001   23.680000   22.280001   22.719999   
           ON           38   47.939999   48.520000   46.040001   47.049999   
           SOL-USD      58  137.620773  148.183838  125.742218  148.030014   
           URI          38  635.500000  643.719971  627.760010  642.320007   
           VST          38  127.779188  134.477620  123.836006  133.429428   

                         Volume    Return  Expected Return  
Date       Asset                                            
2025-01-01 AVAX-USD   334807162       NaN              NaN  
           LINK-USD   670017230       NaN              NaN  
           SOL-USD   2324231668       NaN              NaN  
2025-01-02 ALB          1866100       NaN              NaN  
           AVAX-USD   522067297  0.040742        -0.000056  
...                         ...       ...              ...  
2025-02-28 NCLH        24852400 -0.042159         0.001140  
           ON          13110600 -0.006965        -0.000075  
           SOL-USD   7544673033  0.075637        -0.004608  
           URI           694800  0.014451        -0.000530  
           VST         13060900  0.028075         0.000554  

[528 rows x 8 columns]

Third function to add signal from strategies 

In [ ]:
#previous load data and apply strategies function -> does not work as strategies files have changed 

def load_data_and_apply_strategies(portfolio, strategy_map):
    """
    Function takes in portfolio data (for a date range) and generates signals for HOLD, LONG, SHORT 
    """

    df_list = []
    for asset_name in strategy_map.index:
        strat_file_name = strategy_map.loc[asset_name, "strategy"] 
        strat_class_name = filename_to_classname[strat_file_name]
        
        # Load strategy class dynamically
        strategy_module = importlib.import_module(f'strategies.{strat_file_name}')
        strategy_class = getattr(strategy_module, strat_class_name)

        # Apply strategy to class 
        asset = portfolio.get_asset(asset_name)
        asset_df_with_signals = asset.apply_strategy(strategy_class)

        asset_df_with_signals['Asset'] = asset_name
        df_list.append(asset_df_with_signals.reset_index())
    
    df_with_signals = pd.concat(df_list).set_index(['Date', 'Asset']).sort_index()
    
    return df_with_signals

In [ ]:
df_with_signals = load_data_and_apply_strategies(portfolio, strategy_map)

In [6]:
df_with_signals.head(10)

Open          High           Low  \
Date                      Asset                                               
2023-01-01 00:00:00+00:00 BTC-USD  16547.914062  16630.439453  16521.234375   
2023-01-02 00:00:00+00:00 BTC-USD  16625.509766  16759.343750  16572.228516   
2023-01-03 00:00:00+00:00 BTC-USD  16688.847656  16760.447266  16622.371094   
2023-01-03 00:00:00-05:00 AAPL       128.782649    129.395518    122.742873   
                          JPM        127.603545    129.018846    126.329768   
                          META       122.243862    125.777212    121.706394   
                          MSFT       238.676634    241.298265    233.099519   
                          NVDA        14.838839     14.983721     14.084457   
2023-01-04 00:00:00+00:00 BTC-USD  16680.205078  16964.585938  16667.763672   
2023-01-04 00:00:00-05:00 AAPL       125.431615    127.181276    123.642420   

                                          Close       Volume  EMA_50  EMA_200  \
Date                      Asset                                                 
2023-01-01 00:00:00+00:00 BTC-USD  16625.080078   9244361700     NaN      NaN   
2023-01-02 00:00:00+00:00 BTC-USD  16688.470703  12097775227     NaN      NaN   
2023-01-03 00:00:00+00:00 BTC-USD  16679.857422  13903079207     NaN      NaN   
2023-01-03 00:00:00-05:00 AAPL       123.632530    112117500     NaN      NaN   
                          JPM        127.490311     11054800     NaN      NaN   
                          META       124.154854     35528500     NaN      NaN   
                          MSFT       235.240036     25740000     NaN      NaN   
                          NVDA        14.303278    401277000     NaN      NaN   
2023-01-04 00:00:00+00:00 BTC-USD  16863.238281  18421743322     NaN      NaN   
2023-01-04 00:00:00-05:00 AAPL       124.907707     89113600     NaN      NaN   

                                   RSI  ATR  Lower_Band  ...  %K  %D  MACD  \
Date                      Asset                          ...                 
2023-01-01 00:00:00+00:00 BTC-USD  NaN  NaN         NaN  ... NaN NaN   NaN   
2023-01-02 00:00:00+00:00 BTC-USD  NaN  NaN         NaN  ... NaN NaN   NaN   
2023-01-03 00:00:00+00:00 BTC-USD  NaN  NaN         NaN  ... NaN NaN   NaN   
2023-01-03 00:00:00-05:00 AAPL     NaN  NaN         NaN  ... NaN NaN   NaN   
                          JPM      NaN  NaN         NaN  ... NaN NaN   NaN   
                          META     NaN  NaN         NaN  ... NaN NaN   NaN   
                          MSFT     NaN  NaN         NaN  ... NaN NaN   NaN   
                          NVDA     NaN  NaN         NaN  ... NaN NaN   NaN   
2023-01-04 00:00:00+00:00 BTC-USD  NaN  NaN         NaN  ... NaN NaN   NaN   
2023-01-04 00:00:00-05:00 AAPL     NaN  NaN         NaN  ... NaN NaN   NaN   

                                   MACDh_12_26_9  Signal           OBV  \
Date                      Asset                                          
2023-01-01 00:00:00+00:00 BTC-USD            NaN     NaN  9.244362e+09   
2023-01-02 00:00:00+00:00 BTC-USD            NaN     NaN  2.134214e+10   
2023-01-03 00:00:00+00:00 BTC-USD            NaN     NaN  7.439058e+09   
2023-01-03 00:00:00-05:00 AAPL               NaN     NaN  1.121175e+08   
                          JPM                NaN     NaN  1.105480e+07   
                          META               NaN     NaN  3.552850e+07   
                          MSFT               NaN     NaN  2.574000e+07   
                          NVDA               NaN     NaN  4.012770e+08   
2023-01-04 00:00:00+00:00 BTC-USD            NaN     NaN  2.586080e+10   
2023-01-04 00:00:00-05:00 AAPL               NaN     NaN  2.012311e+08   

                                   TotalSignal  index  EMASignal  AvgVolume10  
Date                      Asset                                                
2023-01-01 00:00:00+00:00 BTC-USD            0    0.0        1.0          NaN  
2023-01-02 00:00:00+00:00 BTC-USD            0    1.0  

In [ ]:
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [ ]:
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [ ]:
def auto_trade(df, strategy_map, strat_df, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(['date', 'asset'])

    assets = df.index.get_level_values(1).unique()
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {'units': 0, 'entry_price': None} for asset in assets}
    cash = config['starting_cash']
    trade_log = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )

        # Calculate thresholds
        cash_floor = config['cash_floor_pct'] * total_value
        #cash_ceiling = config['cash_ceiling_pct'] * total_value
    
        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row['Close']
            signal = row['TotalSignal']
            asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]['units'] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config['buy_pct'] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            if portfolio[asset]['units'] > 0:
                entry_price = portfolio[asset]['entry_price']
                profit_pct = (current_price - entry_price) / entry_price
                if profit_pct >= config['tp_pct'] or profit_pct <= -config['sl_pct']: #IMPLEMENT DYNAMIC TP AND SL
                    if asset_type == 'Crypto':
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]['units'] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]['units'] * current_price)
                        proceeds = portfolio[asset]['units'] * current_price - fee
                    cash += proceeds
                    trade_log.append({
                        'asset': asset, 'date': date,
                        'action': 'SELL_TP' if profit_pct >= config['tp_pct'] else 'SELL_SL', 
                        'price': current_price, 'units': portfolio[asset]['units']
                    })
                    portfolio[asset] = {'units': 0, 'entry_price': None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]['units'] > 0:
                if asset_type == 'Crypto':
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]['units'] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]['units'] * current_price)
                    proceeds = portfolio[asset]['units'] * current_price - fee
                cash += proceeds
                trade_log.append({
                    'asset': asset, 'date': date, 'action': 'SELL_SHORT',
                    'price': current_price, 'units': portfolio[asset]['units']
                })
                portfolio[asset] = {'units': 0, 'entry_price': None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get('Expected Return', None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

            # Check if enough cash remains after purchase
                if buy_amount > 0 and (cash - buy_amount) >= cash_floor:
                    if asset_type == 'Crypto':
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price
                
                    old_units = portfolio[asset]['units']
                    old_price = portfolio[asset]['entry_price']

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price) / total_units
                        if old_units > 0 else current_price
                    ) #update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]['units'] = total_units
                    portfolio[asset]['entry_price'] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append({
                        'asset': asset, 'date': date, 'action': 'BUY_ADD' if old_units > 0 else 'BUY_NEW',
                        'price': current_price,'units': new_units
                    })
                
    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]['units'] * df.loc[(dates[-1], asset), 'Close']
        for asset in assets if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config['starting_cash']

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Export to CSV
        trade_df.to_csv("trade_log.csv", index=False)
        print("Trade log exported to 'trade_log.csv'")

    else:
        print("\n(No trades were executed during the period.)")

    return trade_df, final_value            

## Previous functions 

In [ ]:
def load_data_and_apply_strategies(price_path, strategy_map_path):
    # 1. Load price data
    df_prices = pd.read_csv(price_path, parse_dates=['date'])

    # 2. Load asset-strategy mapping
    strat_df = pd.read_csv(strategy_map_path)

    # 3. Prepare DataFrame (MultiIndex)
    df_prices = df_prices.set_index(['date', 'asset']).sort_index()

    # 4. Dynamically load and apply strategies
    df_list = []
    strategy_instances = {}

    for _, row in strat_df.iterrows():
        asset = row['asset']
        strat_class_name = row['strategy']
        module_name = strat_class_name.lower()  # assumes strategy06.py for Strategy06 -- can be adjusted later based on our new naming

        # Load strategy class dynamically
        strategy_module = importlib.import_module(f'strategies.{module_name}')
        strategy_class = getattr(strategy_module, strat_class_name)
        strategy_instance = strategy_class()
        strategy_instances[asset] = strategy_instance

        # Apply strategy to that asset's data
        asset_df = df_prices.xs(asset, level='asset').copy()
        asset_df = strategy_instance.generate_signals(asset_df)
        asset_df['asset'] = asset
        df_list.append(asset_df.reset_index())

    # 5. Combine all assets back into one MultiIndex DataFrame
    full_df = pd.concat(df_list).set_index(['date', 'asset']).sort_index()

    return full_df, strat_df, strategy_instances

In [ ]:
#Gold
# ---------- GOLD Allocation ---------- only when cash reserve is higher than cash ceiling
        #Limitation: not converting gold to cash real-time if need to buy assets - missing opportunities
        if 'GOLD' in assets and date >= dates[15] and cash > cash_ceiling and (date, 'GOLD') in df.index:
            gold_row = df.loc[(date, 'GOLD')]
            gold_price = gold_row['Close']
            gold_signal = gold_row['TotalSignal']

            # Check TP/SL for gold
            if portfolio['GOLD']['units'] > 0:
                entry_price = portfolio['GOLD']['entry_price']
                profit_pct = (gold_price - entry_price) / entry_price
                if profit_pct >= config['tp_pct'] or profit_pct <= -config['sl_pct']:
                    proceeds = portfolio['GOLD']['units'] * gold_price * (1 - config['fee'])
                    cash += proceeds
                    trade_log.append({
                        'asset': 'GOLD',
                        'date': date,
                        'action': 'SELL_TP/SL_GOLD',
                        'price': gold_price,
                        'units': portfolio['GOLD']['units']
                    })
                    portfolio['GOLD'] = {'units': 0, 'entry_price': None}

            # SELL if signal is SHORT
            if gold_signal == 1 and portfolio['GOLD']['units'] > 0:
                proceeds = portfolio['GOLD']['units'] * gold_price * (1 - config['fee'])
                cash += proceeds
                trade_log.append({
                    'asset': 'GOLD',
                    'date': date,
                    'action': 'SELL_GOLD',
                    'price': gold_price,
                    'units': portfolio['GOLD']['units']
                })
                portfolio['GOLD'] = {'units': 0, 'entry_price': None}

            # BUY gold if LONG signal and cash > ceiling
            elif gold_signal == 2:
                excess_cash = cash - cash_ceiling
                invest_amount = min(excess_cash, 0.10 * total_value) #CAN CHANGE THE 0.10

                if invest_amount > 0 and (cash - invest_amount) >= cash_floor:
                    new_units = (invest_amount * (1 - config['fee'])) / gold_price
                    old_units = portfolio['GOLD']['units']
                    old_price = portfolio['GOLD']['entry_price']

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * gold_price) / total_units
                        if old_units > 0 else gold_price
                    )

                    portfolio['GOLD']['units'] = total_units
                    portfolio['GOLD']['entry_price'] = new_avg_price
                    cash -= invest_amount

                    trade_log.append({
                        'asset': 'GOLD',
                        'date': date,
                        'action': 'BUY_GOLD',
                        'price': gold_price,
                        'units': new_units
                    })